building transformer block - contains all modules (layernorm, gelu, MHA, feedforward, dropout (optional))

In [1]:
import torch
import torch.nn as nn

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [3]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [4]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

START 

In [13]:
class TransformerBlock( nn.Module ):
    def __init__( self, cfg ):
        super().__init__()
        self.norm1 = LayerNorm( cfg["emb_dim"] )
        self.att = MultiHeadAttention( 
            d_in = cfg["emb_dim"], # MHA uses cfg file for input embeddings
            d_out = cfg["emb_dim"], # output dimensions (768)
            context_length = cfg["context_length"], #how long context is allowed to be
            dropout = cfg["drop_rate"], #whether we want to use dropout or not
            num_heads = cfg["n_heads"] # number of heads
        )
        self.dropout = nn.Dropout( cfg["drop_rate"] )
        self.norm2 = LayerNorm( cfg["emb_dim"] )
        self.feedforward = FeedForward( cfg )
        
    def forward( self, x ):
        
        # the attention chunk in this transformer:
        shortcut = x # saving shortcut connection to add later
        x = self.norm1( x ) # first layer normalization
        x = self.att( x ) # attention
        x = self.dropout( x ) #dropout for shortcut connection
        x = x + shortcut # adding back shortcut
        
        # the feed forward chunk:
        shortcut = x
        x = self.norm2( x ) 
        x = self.feedforward( x ) # calling ff module
        x = self.dropout( x )
        x = x + shortcut 
        
        return x 

In [14]:
# instantiate TransformerBlock
block = TransformerBlock( GPT_CONFIG_124M )

In [15]:
input = torch.randn( 3, 4, 768 )

In [16]:
input

tensor([[[-1.0138, -0.1711, -0.4568,  ..., -0.6732,  2.3760, -1.0971],
         [-0.0602,  0.9086,  0.7634,  ..., -0.9583,  2.0364,  0.9667],
         [ 0.8024,  0.0643,  1.1694,  ...,  1.4197, -0.3639,  0.3450],
         [-1.7229, -0.9951, -0.5675,  ...,  0.3248, -0.9400,  2.7976]],

        [[ 1.3809, -1.2950, -0.2924,  ..., -2.2076, -2.6442, -0.1524],
         [ 1.0291, -0.4634,  0.3811,  ..., -2.4836, -1.5849, -1.9833],
         [ 0.8889, -1.8997, -0.1477,  ..., -1.4576,  0.8378,  1.0037],
         [-1.7425, -0.1870, -1.6205,  ..., -0.0154, -1.2379, -0.9598]],

        [[-1.0369, -1.0602,  1.5803,  ..., -0.9275, -0.2586, -0.6271],
         [-0.1527,  0.0501, -0.0368,  ...,  0.0821, -0.9218, -1.1360],
         [ 0.1633, -1.5593, -1.0229,  ..., -1.0836,  0.1972,  0.0800],
         [-0.9314,  0.3987, -0.9203,  ..., -0.3342, -0.2808,  0.1146]]])

In [17]:
output = block( input )

In [18]:
output.shape

torch.Size([3, 4, 768])

In [19]:
output

tensor([[[-1.3015, -1.3773, -0.1499,  ..., -1.2605,  2.1584, -0.9642],
         [-0.1725,  0.5654,  1.0613,  ..., -0.8141,  2.0971,  0.8889],
         [ 0.6496,  0.0140,  1.2823,  ...,  1.3554, -0.9831,  0.3794],
         [-1.9774, -1.0268,  0.0330,  ...,  0.0037, -1.0331,  2.9050]],

        [[ 1.5048, -1.3047, -0.1233,  ..., -1.7641, -3.3623,  0.3238],
         [ 1.5804, -0.3187,  0.1381,  ..., -2.1514, -1.2517, -2.0375],
         [ 1.2269, -1.5924, -0.1322,  ..., -1.8148,  0.6385,  0.9095],
         [-1.6670, -0.1138, -0.8998,  ...,  0.4397, -1.0041, -0.8487]],

        [[-0.5842, -0.8372,  1.6682,  ..., -0.3074, -0.2215, -0.8528],
         [-0.5488, -0.0338,  0.1306,  ...,  0.7247, -1.0461, -1.1586],
         [-0.1593, -1.3226, -1.3551,  ..., -0.5377,  0.1257, -0.4443],
         [-1.0036,  0.2750, -0.7321,  ...,  0.3979, -0.4079,  0.0376]]],
       grad_fn=<AddBackward0>)